[![image](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/phillipslucas/TheBeans/blob/main/docs/examples/ExampleBeansLab7.ipynb)

In [1]:
import thebeans


In [2]:
#thebeans default map

m = thebeans.Map()
m

Map(center=[48.513, -120.218], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [3]:
#Get layer info from map
m.layers



(TileLayer(attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', base=True, max_zoom=19, min_zoom=1, name='OpenStreetMap.Mapnik', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://tile.openstreetmap.org/{z}/{x}/{y}.png'),)

In [4]:
#Change Basemap
#the beans allows passing text directly bypassing ipyleaflet formatting for stock ipyleaflet basemaps
m.add_basemap("Strava.Run")
#m.add_basemap("NASAGIBS.ViirsEarthAtNight2012")
m


Map(center=[48.513, -120.218], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [5]:
#thebeans also allows passing just a URL as a tile layer

url = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}"
m.add_tile_layer(url, "Google Hybrid")
m

Map(center=[48.513, -120.218], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zo…

In [6]:
m.layers

(TileLayer(attribution='&copy; <a href="https://www.openstreetmap.org/copyright">OpenStreetMap</a> contributors', base=True, max_zoom=19, min_zoom=1, name='OpenStreetMap.Mapnik', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://tile.openstreetmap.org/{z}/{x}/{y}.png'),
 TileLayer(name='Strava.Run', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://heatmap-external-a.strava.com/tiles/run/bluered/{z}/{x}/{y}.png'),
 TileLayer(name='Google Hybrid', options=['attribution', 'bounds', 'detect_retina', 'max_native_zoom', 'max_zoom', 'min_native_zoom', 'min_zoom', 'no_wrap', 'tile_size', 'tms', 'zoom_offset'], url='https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}'))

In [7]:
m.clear_layers() #clears all layers

m.add_basemap("OpenTopoMap")
m

Map(bottom=3030.0, center=[48.513, -120.218], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

In [8]:
#thebeans abides by the geojson visualizer format

#local = r"C:\Users\phill\Downloads\us_regions.geojson"
#url = "https://github.com/opengeos/datasets/releases/download/us/us_regions.geojson"

m.add_geojson('StudyArea.geojson', 
              "Study Area", 
              style = {
                "color": "yellow",
                "weight": 2,
                "opacity": 0.8,
                "dashArray": "9",
})
m

Map(bottom=3030.0, center=[48.513, -120.218], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom…

In [10]:
#add shapefiles
m.add_shp(shp/CedarCubMuck.shp)

NameError: name 'shp' is not defined

In [ ]:
add vector

In [ ]:
add raster

In [11]:
import ee

In [14]:
ee.Authenticate()
ee.Initialize()


In [16]:
import geemap

In [24]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)


img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

PreFire = ee.Filter.date('2021-06-01', '2021-07-08')
PostFirePreSlide = ee.Filter.date('2022-06-01', '2022-07-03')
PostSlide = ee.Filter.date('2022-07-05', '2022-09-30')

desc_change = ee.Image.cat(
    desc.filter(PreFire).mean(),
    desc.filter(PostFirePreSlide).mean(),
    desc.filter(PostSlide).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(PreFire).mean(),
    asc.filter(PostFirePreSlide).mean(),
    asc.filter(PostSlide).mean(),
)

m = geemap.Map()
m.set_center(48.5,-120.2, 6)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m.add_geojson('StudyArea.geojson')

m


Map(center=[-120.2, 48.5], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…